# Mini Batches
Iterate over a data stream in mini batches

In [1]:
import pandas as pd
from river import metrics
from river.datasets import Phishing
from river_torch.classification import Classifier
import torch

In [2]:
n_samples = 10000
dataset = Phishing()

metric = metrics.Accuracy()

def build_fn(n_features, n_classes=1):
    net = torch.nn.Sequential(
        torch.nn.Linear(n_features, n_classes),
        torch.nn.Sigmoid()
    )
    return net

optimizer_fn = torch.optim.SGD
model = Classifier(build_fn=build_fn,loss_fn="binary_cross_entropy",optimizer_fn=optimizer_fn)

In [4]:
from itertools import islice

def batcher(iterable, batch_size):
    iterator = iter(iterable)
    while batch := list(islice(iterator, batch_size)):
        yield batch

In [10]:
for batch in batcher(dataset,5):
    x,y = zip(*batch)
    x = pd.DataFrame(x)
    y = list(y)
    y_pred = model.predict_proba_many(x)
    print(y_pred)# make a prediction
    _ = [metric.update(y_i, y_pred[idx]) for idx, y_i in enumerate(y)]# update the metric
    model = model.learn_many(x, y)    # make the model learn
print(f'Accuracy: {metric.get()}')

[{}, {}, {}, {}, {}]


TypeError: unhashable type: 'dict'

In [7]:
list(y)

[True, True, True, True, False]